# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : High dimensional problems
## Chapter : Minimal path for a rolling ball

In this notebook, we present a numerical solution to the rolling ball problem: find the optimal way to roll a ball on a surface, so that it reaches a given position and orientation. The main properties of this problem are as follows:
* *High dimensional.* The configuration space of the system is 5 dimensional: three dimensions for the orientation of the ball, and two dimensions for its position on the surface. Arguably, 5 dimensions is (slightly past) the threshold where discretizations on uniform grids loose their relevance.
* *Strongly anisotropic* The problem is of sub-Riemannian type, and its anisotropy is fully generic, not aligned with the axes of the discretization grid.

The rolling ball problem admits numerous variants. One may for instance roll a ball along a one dimensional line, or on a more complex surface such as another ball.

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..")

In [2]:
from agd import AutomaticDifferentiation as ad
from agd import LinearParallel as lp
from agd import Metrics
from agd import FiniteDifferences as fd
norm = ad.Optimization.norm

In [3]:
import numpy as np; xp=np
import scipy.linalg
from scipy.integrate import odeint
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%5.3g" % x))

## 1. Description of the models

In [4]:
ball_state = np.array([1,2,3])/4 # Some arbitrary state
line_state  = np.concatenate((np.zeros(1),ball_state)) # Some arbitrary line-ball state
plane_state = np.concatenate((np.zeros(2),ball_state)) # Some arbitrary plane-ball state

### 1.1 Conversions

**Rotations and unit quaternions**

In [5]:
def rotation_from_quaternion(q):
    """Produces the rotation associated with a unit quaternion."""
    qr,qi,qj,qk = q
    identity = fd.as_field(xp.eye(3),qr.shape,depth=2)
    return identity + 2*ad.array([
        [-(qj**2+qk**2), qi*qj-qk*qr, qi*qk+qj*qr],
        [qi*qj+qk*qr, -(qi**2+qk**2), qj*qk-qi*qr],
        [qi*qk-qj*qr, qj*qk+qi*qr, -(qi**2+qj**2)]])
def quaternion_from_rotation(r):
    """Produces the unit quaternion, with positive real part, associated with a rotation."""
    qr = np.sqrt(lp.trace(r)+1.)/2.
    qi = (r[2,1]-r[1,2])/(4*qr)
    qj = (r[0,2]-r[2,0])/(4*qr)
    qk = (r[1,0]-r[0,1])/(4*qr)
    return ad.array([qr,qi,qj,qk])

**Euclidean space and quaternions**

In [89]:
def quaternion_from_euclidean(e):
    """Produces a point in the unit sphere by projecting a point in the equator plane."""
    e2 = lp.dot_VV(e,e)
    return ad.array([1.-e2,*(2*e)])/(1.+e2)
def euclidean_from_quaternion(q):
    """Produces a point in the equator plane from a point in the unit sphere."""
    e2 = (1-q[0])/(1+q[0])
    return q[1:]*((1+e2)/2.)

In [7]:
def euclidean_from_rotation(r): return euclidean_from_quaternion(quaternion_from_rotation(r))
def rotation_from_euclidean(e): return rotation_from_quaternion(quaternion_from_euclidean(e))

In [8]:
def check(e):
    q  = quaternion_from_euclidean(e)
    r  = rotation_from_quaternion(q)
    q_ = quaternion_from_rotation(r)
    e_ = euclidean_from_quaternion(q_)
    assert np.allclose( norm(q,axis=0), 1.)
    assert np.allclose(lp.dot_AA(lp.transpose(r),r),np.eye(3))
    if norm(e,axis=0)<1: 
        assert np.allclose(q,q_)
        assert np.allclose(e,e_)

In [66]:
check(ball_state)

**Antisymmetric matrices and their exponentials**

In [10]:
def antisym(a,b,c):
    z=np.zeros_like(a)
    return ad.array([[z, -c, b], [c, z, -a], [-b, a, z]])
def exp_antisym(a,b,c):
    """Matrix exponential of antisym(a,b,c)"""
    s = ad.asarray(a**2+b**2+c**2)
    s[s==0]=1e-20 # Same trick as in numpy's sinc function ...
    sq = np.sqrt(s)
    co,si = np.cos(sq),np.sin(sq)
    cosc,sinc = (1-co)/s,si/sq    
    return ad.array([
        [co+cosc*a**2, cosc*a*b-sinc*c, cosc*a*c+sinc*b],
        [cosc*a*b+sinc*c, co+cosc*b**2, cosc*b*c-sinc*a],
        [cosc*a*c-sinc*b, cosc*b*c+sinc*a, co+cosc*c**2]])

In [11]:
assert np.allclose(exp_antisym(1,2,3),scipy.linalg.expm(antisym(1,2,3)))

### 1.2 Action of a control on a state

In this notebook, controls are antisymmetric matrices described by their coordinates in $R^3$. They act additively on the physical state, and multiplicatively (through their exponential) on the angular state. 

The action of a constant control during unit time is explicit, hence does not need to be numerically integrated. In order to recover the flow, one may simply apply an infinitesimal control during a unit time. The flow may also be obtained directly, see the end of this subsection.

In [12]:
def advance(state,control):
    """Move from a state to another by applying a control during a unit time"""
    state,control = map(ad.asarray,(state,control))
    state_physical = state[:-3]
    state_physical = state_physical + 0.25*control[:len(state_physical)] # Additive action on the physical state
    
    state_angular = rotation_from_euclidean(state[-3:])
    state_angular = lp.dot_AA(state_angular,exp_antisym(*control)) # Left invariant action
    
    return np.concatenate([state_physical,euclidean_from_rotation(state_angular)],axis=0)

In [91]:
δ = ad.Dense.identity(constant=0.)
advance([100,0,0],[δ,0,0]) # Issue : we are switching hemispheres...
#rotation_from_euclidean(np.array([1,0,0]))

(denseAD(array([-0.01,     0,     0]),
 array([[ 0.25],
        [    0],
        [    0]])),)

Close to the origin, the three controls act independently on each coordinate of the euclidean parametrization of the sphere.

In [13]:
δ = ad.Dense.identity(constant=0.)
advance([0,0,0],[δ,0,0])

denseAD(array([    0,     0,     0]),
array([[ 0.25],
       [    0],
       [    0]]))

In [14]:
advance([0,0,0],[0,δ,0])

denseAD(array([    0,     0,     0]),
array([[    0],
       [ 0.25],
       [    0]]))

In [15]:
advance([0,0,0],[0,0,δ])

denseAD(array([    0,     0,     0]),
array([[    0],
       [    0],
       [ 0.25]]))

They also act on the plane or line parametrization, if present.

In [16]:
advance([0,0,0,0,0],[δ,0,0])

denseAD(array([    0,     0,     0,     0,     0]),
array([[ 0.25],
       [    0],
       [ 0.25],
       [    0],
       [    0]]))

In [17]:
advance([0,0,0,0,0],[0,δ,0])

denseAD(array([    0,     0,     0,     0,     0]),
array([[    0],
       [ 0.25],
       [    0],
       [ 0.25],
       [    0]]))

More complex motions, known as commutators, are needed to control the sphere and the plane independently, or to control the sphere using only two of the three available controls. 

Before turning to these more complex motions, we provide, as promised, the direct flow expression.

In [18]:
def flow(state,control):
    """Flow vector for a given state and control"""
    state,control = map(ad.asarray,(state,control))
    state_physical = state[:-3]
    flow_physical = 0.25*control[:len(state_physical)] # Additive action on the physical state
    
    state_angular = rotation_from_euclidean(state[-3:])
    flow_angular = lp.dot_AA(state_angular,antisym(*control)) # Left invariant action

    ϵ = ad.Dense.identity(constant=0.)    
    rec_ad = ad.is_ad(state_angular) or ad.is_ad(flow_angular) # Recursve AD requires a little bit of care
    if rec_ad: state_angular,flow_angular = map(ad.disassociate,(state_angular,flow_angular))
    flow_angular_eucl = euclidean_from_rotation(state_angular+ϵ*flow_angular).gradient(0) # Differentiate the reverse mapping
    if rec_ad: state_angular,flow_angular = map(ad.associate,(state_angular,flow_angular))
    
    return np.concatenate([flow_physical,flow_angular_eucl],axis=0)

In [19]:
assert np.allclose(advance(plane_state,δ*ad.array([1,2,3])).gradient(0),flow(plane_state,[1,2,3]))

### 1.3 Commutators

In [20]:
def commute(state,control1,control2):
    state,control1,control2 = map(ad.asarray,(state,control1,control2))
    state = advance(state, control1)
    state = advance(state, control2)
    state = advance(state,-control1)
    state = advance(state,-control2)
    return state 

On the sphere, the commutator of the first two controls is the third control. This allows to bring the sphere from any state to any other state using only two controls, albeit rather inefficiently since the motion along the third control generated by a commutator is of second order.

In [21]:
δ = ad.Dense2.identity(constant=0.)
commute([0,0,0],[δ,0,0],[0,δ,0]) # Motion generated by a commutator of the first two controls

denseAD2(array([    0,     0,     0]),
array([[    0],
       [    0],
       [    0]]),
array([[[    0]],

       [[    0]],

       [[  0.5]]]))

In [22]:
advance([0,0,0],[0,0,δ**2]) # Same motion generated by the third control, at second order.

denseAD2(array([    0,     0,     0]),
array([[    0],
       [    0],
       [    0]]),
array([[[    0]],

       [[    0]],

       [[  0.5]]]))

Likewise, on the sphere, the commutator of the second and third control is minus the first one.

In [23]:
commute([0,0,0],[0,δ,0],[0,0,δ]) # Motion generated by a commutator of the second and third controls

denseAD2(array([    0,     0,     0]),
array([[    0],
       [    0],
       [    0]]),
array([[[  0.5]],

       [[    0]],

       [[    0]]]))

However, on the plane, which is a vector space, all controls commute. This observation allows to control the motion of the sphere without changing its position on the plane.

In [24]:
commute([0,0,0,0,0],[0,δ,0],[0,0,δ]) # Motion generated by a commutator of the second and third controls.
# Only the sphere is affected

denseAD2(array([    0,     0,     0,     0,     0]),
array([[    0],
       [    0],
       [    0],
       [    0],
       [    0]]),
array([[[    0]],

       [[    0]],

       [[  0.5]],

       [[    0]],

       [[    0]]]))

In [25]:
advance([0,0,0,0,0],[δ**2,0,0]) # Motion generated by the first control, at second order. 
# Both the sphere and plane are affected

denseAD2(array([    0,     0,     0,     0,     0]),
array([[    0],
       [    0],
       [    0],
       [    0],
       [    0]]),
array([[[  0.5]],

       [[    0]],

       [[  0.5]],

       [[    0]],

       [[    0]]]))

The rank of a sub-Riemannian model is the depth of the control commutators needed to generate the entire tangent space.
A rank-0 model is amounts to Riemannian manifold. The numerical simulation, using PDE techniques, of models of rank larger than or equal to two is novel to the knowledge of the author.

### 1.3 Ball models

We present the hamiltonians of the different models, in the form of the symmetric semi-definite matrix defining the dual metric, which makes sense since the models are sub-Riemannian. 

<!---
A more classical presentation, involving the position and impulsion variables, is presented in next section.
--->

In [26]:
def make_hamiltonian(controls,advance=advance):
    """Produces the hamiltonian function associated to a sub-Riemannian model, 
    defined by its controls and the advance function"""
    def hamiltonian(state):
        """The hamiltonian, a quadratic form on the co-tangent space"""
        # Array formatting to apply to several states simultanously
        state=ad.asarray(state); controls_ = fd.as_field(controls,state.shape[1:],depth=1) 
        
        grad = advance(state,controls_).gradient()
        return lp.dot_AA(lp.transpose(grad),grad)
    return hamiltonian

In [27]:
δ = 4*ad.Dense.identity(constant=[0.,0.,0.]) # Three independent symbolic perturbations

The **Riemannian ball** is governed by the three controls, with the same strength on each axis, and its state space is the ball alone. It is thus a Riemannian model. At the origin, the Hamiltonian is the identity matrix.

In [28]:
H_Riemann = make_hamiltonian(δ)
H_Riemann([0,0,0])

array([[    1,     0,     0],
       [    0,     1,     0],
       [    0,     0,     1]])

At any point in space, this hamiltonian is a positive definite matrix, whose inverse is the Riemannian metric. Because of our choice of coordinates, this metric is *conformal*, in other words proportional to the euclidean metric (up to machine precision).

In [29]:
H_Riemann(ball_state)

array([[ 3.52, 5.79e-13, 8.6e-13],
       [5.79e-13,  3.52, 1.73e-12],
       [8.6e-13, 1.73e-12,  3.52]])

The **Reeds shepp ball** is governed only two of the three controls, missing the middle one, and is a sub-Riemannian model of rank one. This model can also be regarded as describing a wheelchair-like vehicle moving on a ball.

In [30]:
H_ReedsShepp = make_hamiltonian([δ[0],0,δ[2]]) # The y-axis rotation control is lacking
H_ReedsShepp([0,0,0])

array([[    1,     0,     0],
       [    0,     0,     0],
       [    0,     0,     1]])

At any point in space, this hamiltonian is a positive semi-definite matrix, with two non-zero eigenvalues.

In [31]:
ham = H_ReedsShepp(ball_state)
ham, np.linalg.eigvalsh(ham)

(array([[ 1.95, 0.781,  1.56],
        [0.781,  3.13, -0.781],
        [ 1.56, -0.781,  1.95]]),
 array([2.64e-16,  3.52,  3.52]))

The **torqueless ball** is governed by two of the three controls, missing the last one. On the ball alone, this model is identical to the Reeds-Shepp model. However the situation changes when put on a plane, as discussed below.

In [32]:
H_Torqueless = make_hamiltonian([δ[0],δ[1],0])
H_Torqueless([0,0,0])

array([[    1,     0,     0],
       [    0,     1,     0],
       [    0,     0,     0]])

In [33]:
ham = H_Torqueless(ball_state)
ham, np.linalg.eigvalsh(ham)

(array([[ 1.62, -0.344, -1.72],
        [-0.344,  3.45, -0.312],
        [-1.72, -0.312,  1.95]]),
 array([-1.11e-16,  3.52,  3.52]))

### 1.4 Ball with support

The same *hamiltonian* functions can be reused, since the advance function handles the supports as well.

The **Riemannian ball on a plane** is a rank one sub-Riemannian model. The ball may be rolled along both the $x$-axis and $y$-axis, and its rotation around the $z$-axis is also controlled.

The hamiltonian is degenerate, as expected, with two vanishing eigenvalues due to the fact that motion in the plane is tied to the rotation of the ball.

In [34]:
ham = H_Riemann(np.zeros(5))
ham,np.linalg.eigvalsh(ham)

(array([[    1,     0,     1,     0,     0],
        [    0,     1,     0,     1,     0],
        [    1,     0,     1,     0,     0],
        [    0,     1,     0,     1,     0],
        [    0,     0,     0,     0,     1]]),
 array([    0,     0,     1,     2,     2]))

The situation is the same at any point in space.

In [35]:
ham = H_Riemann(plane_state)
ham,np.linalg.eigvalsh(ham)

(array([[    1,     0,  0.25,  1.75, -0.625],
        [    0,     1, -1.25, 0.625,  1.25],
        [ 0.25, -1.25,  3.52, 5.79e-13, 8.6e-13],
        [ 1.75, 0.625, 5.79e-13,  3.52, 1.73e-12],
        [-0.625,  1.25, 8.6e-13, 1.73e-12,  3.52]]),
 array([-8e-17, 7.45e-17,  3.52,  4.52,  4.52]))

The **Reeds-Shepp ball on a line** is a rank two sub-Riemannian model. The ball may only be rolled along the $x$-axis, and rotated around the $z$-axis.
In order to generate a rotation around the $y$-axis, a commutator of these two motions is required. In order to generate a rotation around the $x$-axis, without any motion in the plane, a second order commutator is needed involving rotation around the $y$-axis and a rotation around the $z$-axis.

In [36]:
ham = H_ReedsShepp(np.zeros(4))
ham,np.linalg.eigvalsh(ham)

(array([[    1,     1,     0,     0],
        [    1,     1,     0,     0],
        [    0,     0,     0,     0],
        [    0,     0,     0,     1]]),
 array([    0,     0,     1,     2]))

In [37]:
ham = H_ReedsShepp(line_state)
ham,np.linalg.eigvalsh(ham)

(array([[    1,  0.25,  1.75, -0.625],
        [ 0.25,  1.95, 0.781,  1.56],
        [ 1.75, 0.781,  3.13, -0.781],
        [-0.625,  1.56, -0.781,  1.95]]),
 array([-3.37e-16, 4.61e-16,  3.52,  4.52]))

The **torqueless ball on a plane** is a rank two sub-Riemannian model. The ball may be rolled along the $x$-axis and the $y$-axis, but not rotated around the $z$-axis (think of a punctual control, without any torque). In order to generate a rotation around the $z$-axis, a commutator of the rolling motions along the $x$-axis and the $y$-axis is needed. In order to generate a pure rotation around the $x$-axis or the $y$-axis, one needs a commutator of the pure rotation around the $z$-axis, and of the rolling motion along the $x$ or the $y$ axis.

In [38]:
ham = H_Torqueless(np.zeros(5))
ham,np.linalg.eigvalsh(ham)

(array([[    1,     0,     1,     0,     0],
        [    0,     1,     0,     1,     0],
        [    1,     0,     1,     0,     0],
        [    0,     1,     0,     1,     0],
        [    0,     0,     0,     0,     0]]),
 array([    0,     0,     0,     2,     2]))

In [39]:
ham = H_Torqueless(plane_state)
ham,np.linalg.eigvalsh(ham)

(array([[    1,     0,  0.25,  1.75, -0.625],
        [    0,     1, -1.25, 0.625,  1.25],
        [ 0.25, -1.25,  1.62, -0.344, -1.72],
        [ 1.75, 0.625, -0.344,  3.45, -0.312],
        [-0.625,  1.25, -1.72, -0.312,  1.95]]),
 array([-3.67e-16, 7.34e-17, 5.62e-16,  4.52,  4.52]))

## 2. Geodesic shooting

We compute, for checking their consistency, some geodesics of the considered models model. They correspond to the free motion of the corresponding physical system, without external force or obstacles.

In [40]:
# Some arbitrary initial impulsions
ball_impulsion = np.array([4,5,6])/3
plane_impulsion = np.concatenate([[2,3],ball_impulsion],axis=0)
line_impulsion = np.concatenate([[3],ball_impulsion],axis=0)

The `Metrics.Hamiltonian` class provides a few helper utilities, for differentiating hamiltonians.

In [41]:
def class_hamiltonian(H):
    def H_(q,p): return 0.5*lp.dot_VAV(p,H(q),p) # Usual two variables hamiltonian
    return Metrics.Hamiltonian(H_,disassociate_ad=True) # Disassociate because needed for recursive AD

In [42]:
h_Riemann,h_ReedsShepp,h_Torqueless = map(class_hamiltonian,(H_Riemann,H_ReedsShepp,H_Torqueless))

In [51]:
h_Riemann.flow(ball_state,ball_impulsion) # The symplectic flow DpH,-DqH

(array([ 4.69,  5.86,  7.03]), array([-8.02,   -16, -24.1]))

In [52]:
Tint = np.linspace(0,2.*np.pi)
ball_ode = odeint(h_Riemann.flow_cat,np.concatenate((ball_state,ball_impulsion)),Tint)

/Users/mirebeau/opt/miniconda3/envs/agd-hfm_dev/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [82]:
δ = ad.Dense.identity(constant=0.)
advance([1.,0.,0.],[δ,0.,0.])

denseAD(array([  nan,   nan,   nan]),
array([[  nan],
       [  nan],
       [  nan]]))

In [55]:
h_Riemann.flow_cat(np.array([ 0.28, 0.538, 0.795,  1.28,  1.57,  1.85]))

array([ 5.12,  6.28,   7.4, -59.3,  -114,  -168])

In [71]:
H_Riemann(100*np.array([ 0.28, 0.538, 0.795]))

array([[    1, 2.65e-18, -1.73e-18],
       [2.65e-18,     1,     0],
       [-1.73e-18,     0,     1]])

In [74]:
rotation_from_euclidean(100*np.array([ 0.28, 0.538, 0.795]))

array([[0.999, 0.0319, -0.0213],
       [-0.0317, 0.999, 0.0115],
       [0.0217, -0.0109,     1]])

array([[ 0.25,   0.5,  0.75,  1.33,  1.67,     2],
       [ 0.28, 0.538, 0.795,  1.28,  1.57,  1.85],
       [1.22e-114, 2.21e+232, 2.9e-144, 4.26e+257, 3.17e+180, 7.74e+228],
       [4.19e+228, 5.81e+180, 9.77e+199, 2.45e+198, 9.08e+223, 7.37e-260],
       [1.44e+214, 2.66e+233, 2.15e-259, 1.01e+261, 7.5e+270, 1e-144],
       [3.01e+161, 3.32e+257, 6.62e+265, 1.28e-152, 8.76e+252, 3.45e-85],
       [3.88e-265, 2e+161, 1.95e-153, 8.73e+183, 3.88e-265, 4.7e+180],
       [1.58e-52, 6.69e+151, 3.28e-85, 6.79e+199, 2.16e+233, 7.5e+270],
       [2.09e+241, 5.74e+257, 4.36e-114, 1.67e+243, 2.42e+198, 4.97e+180],
       [1.23e+237, 4.06e-265, 4.96e+180, 3.81e+180, 3.88e-265, 1.82e-152],
       [9.8e-86, 5.72e+228, 4.06e-265, 4.07e-265, 1.07e+224, 4.24e+40],
       [4.04e-265, 2.65e+180,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0],
       [    0,     0,     0

In [44]:
print(ad.disassociate(ball_impulsion).shape)
H_Riemann(ad.disassociate(ball_state)).shape

(3, 1)


(3, 3, 1)

In [45]:
rotation_from_euclidean(ad.disassociate(ball_state)).shape

(3, 3, 1)

In [46]:
ad.disassociate(ball_state).shape

(3, 1)

(array([ 4.69,  5.86,  7.03]), array([-8.02,   -16, -24.1]))

In [48]:
help(Metrics.Hamiltonian)

Help on class Hamiltonian in module agd.Metrics.hamiltonian:

class Hamiltonian(builtins.object)
 |  Hamiltonian(H, shape_free=None, vdim=-1, disassociate_ad=False, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  DpH(self, q, p)
 |      Differentiates the Hamiltonian, w.r.t. impulsion.
 |  
 |  DqH(self, q, p)
 |      Differentiates the Hamiltonian, w.r.t. position.
 |  
 |  H(self, q, p)
 |      Evaluates the Hamiltonian, for a given position and impulsion.
 |  
 |  __init__(self, H, shape_free=None, vdim=-1, disassociate_ad=False, **kwargs)
 |      Inputs:
 |      - H : the hamiltonian, which may be either:
 |              * a metric
 |              * a callable function of two arguments (q,p)
 |              * a pair of callable functions, of two arguments (q,p) for a separable hamiltonian.
 |                      (in that case, may also be scalars or matrices, for quadratic hamiltonians)
 |      - shape_free (optional) : the shape of the position and impulsion
 |      - vdim (opt

In [49]:
ham

array([[    1,     0,  0.25,  1.75, -0.625],
       [    0,     1, -1.25, 0.625,  1.25],
       [ 0.25, -1.25,  1.62, -0.344, -1.72],
       [ 1.75, 0.625, -0.344,  3.45, -0.312],
       [-0.625,  1.25, -1.72, -0.312,  1.95]])